본 서적의 깃허브 및 블로그 주소 </br>
https://bit.ly/transformer-git </br>
https://bit.ly/transformer-home


In [48]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=1)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 597 (delta 10), reused 2 (delta 1), pack-reused 569
Receiving objects: 100% (597/597), 57.73 MiB | 26.90 MiB/s, done.
Resolving deltas: 100% (298/298), done.
Updating files: 100% (118/118), done.
/content/nlp-with-transformers/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.30.2
Using datasets v2.13.1
Using accelerate v0.20.3
Using sentencepiece v0.1.99
Using sacremoses v0.0.41
No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.


In [49]:
# Colab에는 트랜스포머가 설치되어 있지 않으므로 이 코드를 사용해 수동 설치
!pip install transformers

In [50]:
# 트랜스포머에서는 pipeline() 함수를 호출하면서 관심작업 이름을 전달해 파이프라인 객체를 만든다.
from transformers import pipeline

classifier = pipeline("text-classification")

처음 위 코드를 실행하면 파이프라인이 자동으로 허깅페이스 허브에서 모델 가중치를 다운로드 한다.  
허깅 페이스 허브 주소  
https://oreil.ly/zLK11

</br>

파라미터인 text-classification은 감성 분석을 위한 모델을 사용하지만  
다중분류와 다중 레이블 분류도 지원한다.  
  
이제 파이프라인이 있으니 예측을 만들어보자

In [51]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

우선 위와 같은 텍스트를 준비했다.  
이것은 온라인 주문에 대한 가짜 피드백으로 법률 계약, 제품 설명 등의 내용이 들어가 있다.  
이것은 과연 긍정적인 피드백인가 부정적인 피드백인가?  
</br>
한 번 분류해보자

# 텍스트 분류

In [52]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


각 예측은 하나의 파이썬 딕셔너리 이므로 판다스를 사용한다.  
DataFrame 메소드를 통해 결과를 출력해본 결과 부정적이라는 결과가 나온다.  

</br>

자, 이걸 분석해보자

# 개체명 인식

In [53]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


이 파이프라인은 모든 개체명을 감지하고 ORG(조직), LOC(위치), PER(사람)  
같은 카테고리에 할당했다.  
  
Optimus Prime은 두 단어지만 하나의 카테고리에 할당 (MISC(그 외))에 할당했다.  
</br>

점수는 모델이 개체명을 얼마나 확신하는지 나타낸다.  
참고로 word의 ##은 토큰화로, 이에 대해선 추후에 알아보자

</br>
이번엔 질답을 해보자

# 질문 답변

In [54]:
reader = pipeline("question-answering")
question = "What does the customer want?"   # 질문
outputs = reader(question = question, context = text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


질문에 대해서는 딱히 긍정도 부정도 아닌 값이 나왔다.  
</br>

이번엔 장황하게 늘어놓은 글을 요약하는 요약모델에 대해 알아보자

# 요약

In [55]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead. As a lifelong enemy
of the Decepticons, I hope you can understand


다음과 같이 요약되었다.  
Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand  

</br>
요약이 완벽하지는 않지만 불만을 제기한 사람이 Bumblebee라는 것은  
확실하게 구분했다.  
  
</br>
이번엔 번역에 대해 알아보자

# 번역

In [56]:
# 먼저 번역에 필요한 패키지 설치
!pip install sentencepiece

In [57]:
!pip install transformers[sentencepiece]

In [58]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:885 in pipeline       │
│                                                                                                  │
│   882 │   │   │   │   tokenizer_kwargs = model_kwargs.copy()                                     │
│   883 │   │   │   │   tokenizer_kwargs.pop("torch_dtype", None)                                  │
│   884 │   │   │                                                                                  │
│ ❱ 885 │   │   │   tokenizer = AutoTokenizer.from_pretrained(                                     │
│   886 │   │   │   │   tokenizer_identifier, use_fast=use_fast, _from_pipeline=task, **hub_kwar   │
│   887 │   │   │   )                                                                              │
│   888                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:714 in     │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   711 │   │   │   │   if tokenizer_class_py is not None:                                         │
│   712 │   │   │   │   │   return tokenizer_class_py.from_pretrained(pretrained_model_name_or_p   │
│   713 │   │   │   │   else:                                                                      │
│ ❱ 714 │   │   │   │   │   raise ValueError(                                                      │
│   715 │   │   │   │   │   │   "This tokenizer cannot be instantiated. Please make sure you hav   │
│   716 │   │   │   │   │   │   "in order to use this tokenizer."                                  │
│   717 │   │   │   │   │   )                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to 
use this tokenizer.

# ! 왜인지 코드가 작동하지 않는다.
독일어로 번역하는 기능이다.  
이 부분은 스킵

# 텍스트 생성

In [61]:
from transformers import set_seed
set_seed(42) # 동일 결과를 재현하기 위해 지정

In [59]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. You should
contact your local distributor for a replacement of your order.

Thanks for your time, and good evening.

Thank you for your recent order of Optimus Prime action figure.

I am very glad to hear that you were able to contact your local distributor
because I would like to know where your order will take these


시드값은 42로 고정한다. 이는 책의 예제와 같은 결과를 얻기 위함이다.  


---
# 허깅페이스

허깅페이스는 허브와 라이브러리로 구분한다.  
라이브러리는 코드를 제공하고  
허브는 사전 훈련된 모델 가중치, 데이터셋, 평가 지표를 위한 코드 등을 제공한다.

## 허깅페이스 토크나이저
토크나이저는 다양한 토큰화 전략을 제공한다.  
토큰화란 더 작은 단위로 분할하는 것이다.  
</br>

## 허깅페이스 데이터셋
대용량 데이터셋을 다루는 것은 어렵고 번거롭다.  
이런 과정을 허브에서 수천 개의 데이터셋에 대한 표준 인터페이스를 제공해  
단순화 한다.  
스마트 캐싱을 제공하고 메모리 매핑이라는 특별한 메커니즘을 활용해 램 부족을 피한다.  
</br>

## 허깅페이스 액셀러레이트
액셀러레이트는 사용자의 로직을 처리하는 일반적인 훈련 루프에 훈련 인프라에   필요한 추상화 층을 제공한다.  



___
# 트랜스포머의 과제 (또는 한계)

트랜스포머에도 한계와 고쳐나가야할 것들이 있다.  
이는 다음과 같다.  

</br>

### 언어
죄다 영어만 쓴다.  
물론 그 외 언어도 있지만 훈련 모델을 찾기 어렵다.  

</br>

### 데이터 가용성
모델에 필요한 훈련 데이터 양은 전이 학습을 통해 크게 줄일 수 있지만  
사람이 작업을 수행하는 데 필요한 양에 비하면 여전히 많다.  

</br>

### 긴 문서 처리
문서 길이가 문단 정도일 때만 잘 작동하며  
문서가 너무 길어지면 비용이 많이 든다.

</br>

### 불투명성

트랜스포머는 다른 딥러닝 모델처럼 투명하지 않다.  
모델이 그렇게 예측한 이유를 설명하기가 어렵다는 뜻이다.

</br>

### 편향
트랜스포머 모델은 인터넷 텍스트 데이터를 사용해 훈련한다.  
때문에 편향이 발생할 수 밖에 없다.  
인종차별, 성차별 또는 더 나쁜 편향이 있는지 확인하기 어렵다.  

</br>
</br>

이러한 한계는 어느정도 극복이 가능하다.  
앞으로 이러한 것들을 극복해나가는 과정에 대해서 다루어 보겠다.





